# ADAS Object Detection - Training on Google Colab

This notebook trains YOLOv8 on BDD100K dataset using Google Colab's free GPU.

**Runtime**: Select GPU runtime (Runtime > Change runtime type > GPU)

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repository
!git clone https://github.com/Pranav1011/ADAS-OBJECT-DETECTION.git
%cd ADAS-OBJECT-DETECTION

In [ ]:
# Install dependencies
!pip install -q ultralytics opencv-python albumentations wandb

## 2. Download BDD100K Dataset

**Option A**: Upload from Google Drive (recommended)
1. Download BDD100K from https://bdd-data.berkeley.edu/
2. Upload to your Google Drive
3. Mount and copy files

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Copy dataset from Drive (adjust paths as needed)
!mkdir -p data/raw
!cp /content/drive/MyDrive/datasets/bdd100k_images_10k.zip data/raw/
!cp /content/drive/MyDrive/datasets/bdd100k_labels_release.zip data/raw/

In [ ]:
# Extract dataset
!cd data/raw && unzip -q bdd100k_images_10k.zip
!cd data/raw && unzip -q bdd100k_labels_release.zip

## 3. Convert Annotations to YOLO Format

In [ ]:
# Convert BDD100K to YOLO format
!python -m data_pipeline.convert_annotations \
    --bdd-root data/raw \
    --output-dir data/processed

In [ ]:
# Verify dataset
!ls -la data/processed/
!cat data/processed/dataset.yaml

## 4. Train YOLOv8 Model

In [ ]:
from ultralytics import YOLO

# Load YOLOv8m model
model = YOLO('yolov8m.pt')

# Train
results = model.train(
    data='data/processed/dataset.yaml',
    epochs=100,
    batch=16,  # Adjust based on GPU memory (8 for T4, 16 for A100)
    imgsz=640,
    patience=20,
    device=0,
    project='runs/train',
    name='yolov8m-bdd100k',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    lr0=0.001,
    cos_lr=True,
    warmup_epochs=5,
)

## 5. Evaluate Model

In [ ]:
# Evaluate on validation set
metrics = model.val(data='data/processed/dataset.yaml')
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")

## 6. Export Model

In [ ]:
# Export to ONNX
model.export(format='onnx', imgsz=640, simplify=True)

In [ ]:
# Copy best weights to weights folder
!mkdir -p weights
!cp runs/train/yolov8m-bdd100k/weights/best.pt weights/
!cp runs/train/yolov8m-bdd100k/weights/best.onnx weights/

## 7. Save to Google Drive

In [ ]:
# Save trained weights to Google Drive
!cp -r weights /content/drive/MyDrive/adas-detection/
!cp -r runs/train/yolov8m-bdd100k /content/drive/MyDrive/adas-detection/
print("Saved to Google Drive!")

## 8. Quantization (Optional)

Run INT8 quantization for faster inference.

In [ ]:
# Install ONNX Runtime
!pip install -q onnxruntime

# Run quantization
!python -m quantization.onnx_quantize \
    --model weights/best.onnx \
    --calibration-dir data/processed/images/val \
    --num-calibration 500

## 9. TensorRT Optimization (Optional)

In [ ]:
# Export to TensorRT (requires NVIDIA GPU)
model.export(
    format='engine',
    imgsz=640,
    half=True,  # FP16
    device=0
)

## 10. Benchmark Models

In [ ]:
# Benchmark all models
!python -m quantization.benchmark \
    --models-dir weights \
    --runs 100